In [1]:
import pandas as pd  
import numpy as np  
from sklearn.svm import SVR
from sklearn import metrics
import matplotlib.pyplot as plt 
from sklearn.model_selection import GridSearchCV

In [2]:
RDL = pd.read_excel("/Users/moca_lucy/Documents/大學/專題/RDL_AlCu/PCA_newdata(RDL).xlsx", sheet_name="Weekly", header=0)

In [3]:
traindata = RDL[0:48]
testdata = RDL[48:57]
x_columns = ["PC1", "PC2", "PC3", "PC4", "PC5", "PC6"]
x_train = traindata[x_columns]
traindata
y_train = traindata["Y"]
x_test = testdata[x_columns]
y_test = testdata["Y"]
#type(y_train)
testdata

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,Y
48,1044.238487,573.451727,-1079.206943,261.919029,1498.031056,-93.706524,2007.920033,2092.678065,-630.369489,120.076357,4
49,1656.486111,-3.935155,-1303.096474,-926.030366,2161.899032,263.127308,2494.498301,2043.043302,-406.791080,110.768323,4
50,1204.096705,2002.732064,-1740.018732,2487.004736,2012.229292,-783.596193,3287.661338,4317.526802,-1628.876128,252.268450,8
51,1763.092871,1352.946363,-1880.264829,649.747032,2149.109519,-466.529733,3509.418976,3902.377306,-1249.312238,222.402947,11
52,2175.113368,-569.872018,-1447.576774,-1681.732323,3187.806307,724.406911,2990.722740,2021.115460,-205.482824,105.134392,3
53,1965.457152,173.833201,-1582.716270,-179.409282,3253.948657,359.923103,3273.374569,2888.796556,-683.386243,158.189450,7
54,918.544769,1595.627673,-1360.125474,1531.767918,1376.797291,-606.859004,2521.873204,3230.728243,-1193.646856,190.511814,5
55,448.294664,459.217945,-599.666977,453.355470,931.874291,-40.835936,1114.576903,1222.318479,-401.697712,76.718120,5
56,725.558233,645.250019,-877.094035,654.769194,1381.170098,-84.541414,1681.155178,1842.352301,-601.204031,107.355348,2


In [15]:
cls = SVR(kernel="rbf", C=14)
cls.fit(x_train, y_train)
print("the test score of SVR:{:.3f}".format(cls.score(x_train, y_train)))
print("the test score of SVR:{:.3f}".format(cls.score(x_test, y_test)))

the test score of SVR:0.471
the test score of SVR:0.471


In [ ]:
'''parameters = {'kernel': ('linear', 'rbf'), 'C': [1, 2, 4], 'gamma': [0.125, 0.25, 0.5, 1, 2, 4]}
clf = GridSearchCV(cls, param_grid=parameters)
grid_search = clf.fit(x_train, y_train)

print("Best score: %0.3f" % grid_search.best_score_)
print(grid_search.best_estimator_)'''

In [ ]:
#gammas = np.logspace(-2, 1)
#gammas

In [ ]:
#svr_predict = cls.predict(x_test)

In [16]:
def mape(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    sum=0
    denom=0
    for index, value in enumerate(y_true):
        if value:
            sum+=np.abs((y_true[index]-y_pred[index])/y_true[index])
            denom+=1
    return (sum/denom)*100

In [17]:
def mase(y_true, y_pred):
    y_num = y_true.size
    d = np.abs(np.diff(y_true)).sum()/(y_num-1)
    ret = np.abs(y_true - y_pred)
    return ret.mean()/d

In [18]:
def table(lm, x, y, w):
    y_pred = lm.predict(x)
    y_true = y
   # print(y_true[190])
    #print(np.where(np.isnan(y_pred)))
    #print(y_pred, y_true)
    MAE = round(metrics.mean_absolute_error(y_true, y_pred), 4)
    #print(metrics.mean_absolute_error(y_true, y_pred))
    MSE = round(metrics.mean_squared_error(y_true, y_pred), 4)
    RMSE = round(np.sqrt(metrics.mean_squared_error(y_true, y_pred)), 4)
    MAPE =  str(round(mape(y_true, y_pred), 4))+'%'
    MASE = round(mase(y_true, y_pred), 4)
    errortype = ["MAE", "MSE", "RMSE", "MAPE", "MASE"]
    error = [MAE, MSE, RMSE, MAPE, MASE]
    #print(error)
    if w == "train":
        error_dict = {" ":errortype, "training":error}
    elif w == "test":
        error_dict = {" ":errortype, "testing":error}
    #error_dict = {" ":errortype, "誤差分析":error}
    pd.options.display.float_format = '{:.4f}'.format
    df = pd.DataFrame(error_dict)
    df = df.set_index(" ").rename_axis(None)
    #print(df)
    return df

df1 = table(cls, x_train, y_train, "train")
df1

,training
MAE,1.1432
MSE,2.7074
RMSE,1.6454
MAPE,26.1615%
MASE,0.4298


In [19]:
df2 = table(cls, x_test, y_test, "test")
df2

,testing
MAE,1.6891
MSE,3.6580
RMSE,1.9126
MAPE,33.946%
MASE,0.5630
